In [6]:
# Import 
import os
import json
import pandas as pd
import traceback

#Before running this notebook excute your environment this cmd conda install ipykernel 

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

In [8]:
# it takes values  .env from our local folder whatever the values key
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
key=os.getenv("OPENAI_API_KEY")

In [10]:
# Connect opneai to langchain OpenAI
llm=ChatOpenAI(openai_api_key=key,model="gpt-3.5-turbo-0613",temperature=0.5)

e:\Python\New folder\AIprojects\projects\ATSTrackingSystem\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
client=OpenAI(openai_api_key=key)

e:\Python\New folder\AIprojects\projects\ATSTrackingSystem\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [18]:
Template_1="""
Here i have provoided the candidate resume.
you role and responsibility is to find out the below task

1. Find out the how many years of experience needed (e.g : fresher,experinece or in numbers 1,2 3,etc
2. Find out the specific tool if they mentioned any specific tool experienced person
3. Find out if any certificate releated to the job description
4. Find out the skill,programming languages 
5. Find out other information related to job description
6. Find out the candidate qualification
7. Find out the Key Responsibilities:
8. Find out the Requirements:

Here below the resume information
##########################
{candidate_resume}
"""

Resume_template=PromptTemplate(
    input_variables=["candidate_resume"],
    template=Template_1
)

resume_chain=LLMChain(llm=llm,prompt=Resume_template,output_key="candidate_info",verbose=True)

In [19]:
Template="""
Here below i have given the candidate resume information
{candidate_info}

Here i have given the Job description
###################################
{job_description}
Your are the profile matchin expertise.please match these resume and job description 

Perform below task
***********************
1. Find out the how many years of experience needed (e.g : fresher,experinece or in numbers 1,2 3,etc
2. Find out the specific tool if they mentioned any specific tool experienced person
3. Find out if any certificate releated to the job description
4. Find out the skill,programming languages.
6. Find out the candidate qualification 
7. Find out the Key Responsibilities:
8. Find out the Requirements:

Can you tell me the matching percentage of these ?
Can you please match candidate profile to jobdescription
Can you please suggest what need to be imporve



"""
Job_template=PromptTemplate(
    input_variables=["candidate_info","job_description"],
    template=Template

)

job_chain=LLMChain(llm=llm,prompt=Job_template,output_key="job_skill",verbose=True)


In [20]:
# Sequential Chain
generate_evaluate_chain=SequentialChain(chains=[resume_chain,job_chain],
                input_variables=["job_description","candidate_resume"],
                output_variables=["candidate_info","job_skill"])


In [38]:
PercentageTemplate="""

Here we have both candidate and job description

###################################
{job_skill}
Your are the profile matchin expertise.please match these resume and job description 

Perform below task
***********************
1. Find out the how many years of experience needed (e.g : fresher,experinece or in numbers 1,2 3,etc
2. Find out the specific tool if they mentioned any specific tool experienced person
3. Find out if any certificate releated to the job description
4. Find out the skill,programming languages.
6. Find out the candidate qualification 
7. Find out the Key Responsibilities:
8. Find out the Requirements:


Can you give the exact percentage that match with job description?



"""
Job_template=PromptTemplate(
    input_variables=["job_skill"],
    template=PercentageTemplate

)

per_job_chain=LLMChain(llm=llm,prompt=Job_template,output_key="Percentage",verbose=True)


# Sequential Chain
job_generate_evaluate_chain=SequentialChain(chains=[resume_chain,job_chain,per_job_chain],
                input_variables=["job_description","candidate_resume"],
                output_variables=["candidate_info","job_skill","Percentage"])


In [21]:
#Reading the job description
from pypdf import PdfReader
with open("../Notebook/Data/jobdescription.txt","r") as file:
    job_description=file.read()
#print(job_description)

candidate_resume=""

#read file file
reader=PdfReader("../Notebook/Data/Resume.pdf")

for i in range(len(reader.pages)):
    candidate_resume+=reader.pages[i].extract_text()





# """

In [39]:
response=job_generate_evaluate_chain({
    "job_description":job_description,
    "candidate_resume":candidate_resume

}
)



> Entering new LLMChain chain...
Prompt after formatting:

Here i have provoided the candidate resume.
you role and responsibility is to find out the below task

1. Find out the how many years of experience needed (e.g : fresher,experinece or in numbers 1,2 3,etc
2. Find out the specific tool if they mentioned any specific tool experienced person
3. Find out if any certificate releated to the job description
4. Find out the skill,programming languages 
5. Find out other information related to job description
6. Find out the candidate qualification
7. Find out the Key Responsibilities:
8. Find out the Requirements:

Here below the resume information
##########################
 /  
 /  
 /  KARUPPASAMY V
Data Scientist | ML Engineer 
+91 73393 68926 karuppasamy12041998@gmail.com https:/ /kpa yyam1998.github.io/P ortfolio_KP/
Chennai,T amilnadu
SUMMARY
With a solid foundation built upon 2 y ears 3 months softwar e de velopment along  
with hands-on experience in data analysis and machin

In [40]:
print(response.get("Percentage"))

1. The candidate has 2 years and 3 months of software development experience, indicating that they have some experience in the field. (Experience needed: Experience)

2. The candidate has experience with BizTalk, which is not mentioned in the job description. However, they also have experience with C programming language, which is not mentioned in the job description either. (Specific tool: None mentioned)

3. No specific certificates related to the job description are mentioned in the resume. (Certificates: None mentioned)

4. The candidate has skills in Python and SQL programming languages, which are mentioned in the job description. (Skills/programming languages: Python, SQL)

5. Other information related to the job description includes data manipulation, data analysis, data science, and machine learning model development, which align with the job description. (Skills: Data manipulation, data analysis, data science, machine learning model development)

6. The candidate has a Master 

In [22]:
response=generate_evaluate_chain({
    "job_description":job_description,
    "candidate_resume":candidate_resume

}
)



> Entering new LLMChain chain...
Prompt after formatting:

Here i have provoided the candidate resume.
you role and responsibility is to find out the below task

1. Find out the how many years of experience needed (e.g : fresher,experinece or in numbers 1,2 3,etc
2. Find out the specific tool if they mentioned any specific tool experienced person
3. Find out if any certificate releated to the job description
4. Find out the skill,programming languages 
5. Find out other information related to job description
6. Find out the candidate qualification
7. Find out the Key Responsibilities:
8. Find out the Requirements:

Here below the resume information
##########################
 /  
 /  
 /  KARUPPASAMY V
Data Scientist | ML Engineer 
+91 73393 68926 karuppasamy12041998@gmail.com https:/ /kpa yyam1998.github.io/P ortfolio_KP/
Chennai,T amilnadu
SUMMARY
With a solid foundation built upon 2 y ears 3 months softwar e de velopment along  
with hands-on experience in data analysis and machin

In [23]:
print(response.get("job_skill"))

1. The candidate has 2 years and 3 months of software development experience, which meets the minimum requirement of 1 year of experience mentioned in the job description.

2. The candidate has experience with the BizTalk tool, but there is no mention of any specific tool in the job description.

3. There is no mention of any certificates related to the job description in the candidate's resume.

4. The candidate has skills in programming languages such as Python and SQL, which are mentioned in the job description as required skills.

5. The candidate has experience in data manipulation, analysis, data science, and machine learning model development, which aligns with the job description's requirement for expertise in data science, machine learning, and programming.

6. The candidate has a Master of Computer Application degree and a Bachelor of Computer Science degree, which meet the requirement of a B.Tech/B.E./MSc./MBA in engineering, maths, or statistics from a top college mentioned

In [16]:
with open("../Notebook/Data/jobdescription.txt","r") as file:
    job_description=file.read()
print(job_description)

1)Resource must have more than 5+ years of experience in pure development in java.

2) Resource should have experience jdk 1.8 version.

3) Resource should know to write LLD from HLA.

4) Resource should have more than 8+ years of development experience in Java based web application.

5) Must have knowledge on JAX-RS, JAX-WS, JPA, JDBC. REST and SOAP, Spring MVC,Hibernate.

6) Must have known JEE design pattern and its technology.

7) Must have knowledge on Oracle Database (SQL) or MSSQL for more than 4 years

8) Resource should have debugging skill for quick fix.

9) Resource should have very good communication skill to sort and fulfill application need.
"""


Karuppasamy
